In [193]:
from dataclasses import dataclass
import random
from typing import Callable, List
import pandas as pd
import altair as alt


@dataclass
class TestData:
    """Test data class for testing the performance of the algorithm. The data is a list of integers."""

    name: str
    data: List[int]

    def display(self):
        """Display the data using Altair."""
        df = pd.DataFrame(enumerate(self.data), columns=["index", "value"])
        alt.Chart(df).mark_circle().properties(
            title=self.name, width=1200, height=100
        ).encode(
            alt.X("index:Q"),
            alt.Y("value:Q", scale=alt.Scale(zero=True, domain=[0, 0xFFFFFFFF])),
        ).display()

    def transform(
        self, f: Callable[["TestData"], List[int]], name: str = "transform"
    ) -> "TestData":
        """Transform the data using the function f. The function f should take a TestData object and return a list of integers."""
        return TestData(name=f"{name}({self.name})", data=f(self.data))


TEST_DATA_LEN = 512 // 4 # (512 + 8) * 4 + 12
test_datas = [
    TestData(name="All 0", data=[0x00000000 for _ in range(TEST_DATA_LEN)]),
    TestData(name="All 1", data=[0xFFFFFFFF for _ in range(TEST_DATA_LEN)]),
    TestData(name="increment", data=[(i & 0xFFFFFFFF) for i in range(TEST_DATA_LEN)]),
    TestData(
        name="increment2",
        data=[((i * 0x10000000) & 0xFFFFFFFF) for i in range(TEST_DATA_LEN)],
    ),
    TestData(name="bitwalk", data=[(1 << (i % 32)) for i in range(TEST_DATA_LEN)]),
    TestData(
        name="bitwalk2",
        data=[(~(1 << (i % 32)) & 0xFFFFFFFF) for i in range(TEST_DATA_LEN)],
    ),
    TestData(
        name="random1",
        data=[random.randint(0, 0xFFFFFFFF) for _ in range(TEST_DATA_LEN)],
    ),
    TestData(
        name="random2",
        data=[random.randint(0, 0xFFFFFFFF) for _ in range(TEST_DATA_LEN)],
    ),
]

for test_data in test_datas:
    test_data.display()


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

## Bit Scramble

In [207]:
from typing import Generator, List, Optional

class Lfsr32:
    """Linear Feedback Shift Register"""

    def __init__(self, init_value: int = 1, taps: List[int] = [31, 30, 28, 10, 2,1]):
        self.init_value = init_value
        self.value = init_value
        self.taps = taps

    def __tap_bits__(self) -> int:
        return sum([1 << tap for tap in self.taps])

    def reset(self, init_value: Optional[int] = None):
        if init_value is not None:
            init_value = self.init_value
        self.value = self.init_value

    def next(self) -> int:
        tap_bits = self.__tap_bits__()
        self.value = ((self.value >> 1) ^ (-(self.value & 1) & tap_bits)) & 0xFFFFFFFF
        return self.value

    def __str__(self):
        return f"Lfsr32(init={self.init_value}, taps={self.taps})"


def scrample_datas(
    lfsr: Lfsr32,
    data: List[int],
) -> Generator[int, None, None]:
    """Scramble data with LFSR32"""
    lfsr.reset()
    return [d ^ lfsr.next() for d in data]


# test
lfsr = Lfsr32()
scramble_test_datas = [
    test_data.transform(lambda x: scrample_datas(lfsr, x), name=f"xor {lfsr}")
    for test_data in test_datas
]
for test_data in scramble_test_datas:
    test_data.display()

# decode & verify
descramble_test_datas = [
    test_data.transform(lambda x: scrample_datas(lfsr, x), name=f"xor {lfsr}")
    for test_data in scramble_test_datas
]
for test_data in descramble_test_datas:
    test_data.display()
for src, dst in zip(test_datas, descramble_test_datas):
    assert src.data == dst.data




alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

...

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)